# CIFAR10 데이터셋
- '비행기(airplane)', '자동차(automobile)', '새(bird)', '고양이(cat)', '사슴(deer)', '개(dog)', '개구리(frog)', '말(horse)', '배(ship)', '트럭(truck)'의 3채널(컬러), 32x32 이미지와 레이블로 구성(60,000개)
- https:huggingface.co/datasets/cifar10

# Install packages
- pip install -q 옵션 : 더 적은 출력 표시

In [ ]:
!pip install -q transformers datasets
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.8 MB/s eta 0:00:00


In [ ]:
import warnings

warnings.filterwarnings('ignore')

# Loading the data
- datasets.load_dataset(데이터셋 이름, split=['train[:x]', 'test[:y]'])
  - 전체 데이터셋 사이즈 중, train 데이터에서 x개까지, test 데이터에서 y개까지 가져온다.
    - 해당 데아터에 train명으로 분리된 데이터셋과 test명으로 분리된 데이터셋이 이미 존재함
  - train_test_split()을 사용하여 validation set도 구성 가능
    - train과 test 키로 각 데이터셋이 구성됨

In [ ]:
from datasets import load_dataset

train_ds, test_ds = load_dataset('cifar10', split=['train[:5000]', 'test[:2000]'])
splits = train_ds.train_test_split(test_size=0.1)
train_ds = splits['train']
val_ds = splits['test']
train_ds[0].keys()

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/120M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/23.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

dict_keys(['img', 'label'])

# Preprocessing the data
- Vision Transformers는 동일 이미지 사이즈와 동일 채널별 Normalization 시에 성능이 좋음
- ViTFeatureExtractor()를 통해, 해당 Pre-Trained 모델의 학습 시 적용된 config를 확인할 수 있다.
- 채널별 픽셀값은 'pixel_values', 해당 이미지의 분류값은 'labels'에 넣어주면 해당 Pre-Trained 모델로 학습 및 예측 가능

In [ ]:
from transformers import ViTFeatureExtractor

feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
feature_extractor

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

ViTFeatureExtractor {
  "do_convert_rgb": null,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTFeatureExtractor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

# Augmentation
- 적은 데이터를 증강하는 기법으로 이미지 모델에서 성능을 높이는데 기여한 기법
- 또한 다양한 test 데이터에 대해서도 성능을 낼 수 있도록 데이터를 임의로 다양하게 변형하여 학습시키기 위해서도 많이 사용함
- pytorch torchvision에서 제공하는 데이터셋은 데이터 변경을 용이하게 할 수 있도록 몇가지 변형을 제공함
- 주요 함수
  - torchvision.transforms.ToTensor() : PIL 이미지 또하는 ndarray 데이터를 텐서 형태로 변형시켜줌
  - torchvision.transforms.Normalize(mean, std)
    - mean, std는 각 채널별 평균과 표준편차(데이터 정규화를 위한 기법), 텐서에만 적용 가능
    - 예: 3채널 데이터라면,
      - 각 채널의 평균, 표준편차를 0.5로 세팅한다면
      - transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
  - torchvision.transforms.Resize(size)
    - 이미지의 사이즈 변경
    - 예: 이미지를 224x224로 변경하고자 한다면,
      - transforms.Resize((224, 224))
  - torchvision.transforms.Compose()
    - 여러 transform을 하나로 구성하는 기능

In [ ]:
from torchvision import transforms

normalize = transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
transforms_for_train = transforms.Compose(
    [
        transforms.Resize(tuple(feature_extractor.size.values())),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]
)
transforms_for_val = transforms.Compose(
    [
        transforms.Resize(tuple(feature_extractor.size.values())),
        transforms.CenterCrop(tuple(feature_extractor.size.values())),
        transforms.ToTensor(),
        normalize,
    ]
)

def train_transforms(imagedata):
  imagedata['pixel_values'] = [transforms_for_train(image.convert("RGB")) for image in imagedata['img']]
  return imagedata

def test_transforms(imagedata):
  imagedata['pixel_values'] = [transforms_for_val(image.convert("RGB")) for image in imagedata['img']]
  return imagedata

# Set the transforms
train_ds.set_transform(train_transforms)
val_ds.set_transform(test_transforms)
test_ds.set_transform(test_transforms)

print(train_ds[0].keys())
print(type(train_ds[0]['img']))
print(type(train_ds[0]['label']), train_ds[0]['label'])
print(type(train_ds[0]['pixel_values']), train_ds[0]['pixel_values'].shape)

dict_keys(['img', 'label', 'pixel_values'])
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'int'> 1
<class 'torch.Tensor'> torch.Size([3, 224, 224])


# Trainer 활용을 위해 필요한 data_collator 함수
- 인덱스 번호 기반 데이터셋(map-style dataset)을 기반으로 mini_batch 구성 시 샘플을 리스트로 합쳐주는 기능을 구현해야한다.

In [ ]:
from torch.utils.data import DataLoader
import torch

def collate_fn(imagedata):
  pixel_values = torch.stack([example["pixel_values"] for example in imagedata])
  # "pixel_values" 텐서들만 꺼내서 스택으로 쌓음
  labels = torch.tensor([example["label"] for example in imagedata])
  # label 값들을 꺼내서 텐서로 만듦
  return {"pixel_values": pixel_values, "labels": labels}

# DataLoader 사용 시에는 다음과 같이 사용할 수 있다.
# train_dataloader = DataLoader(train_ds, collate_fn=collate_fn, batch_size=16)

# Define the model

In [ ]:
id2label = {id: label for id, label in enumerate(train_ds.features['label'].names)}
label2id = {label: id for id, label in id2label.items()}
id2label

{0: 'airplane',
 1: 'automobile',
 2: 'bird',
 3: 'cat',
 4: 'deer',
 5: 'dog',
 6: 'frog',
 7: 'horse',
 8: 'ship',
 9: 'truck'}

In [ ]:
label2id

{'airplane': 0,
 'automobile': 1,
 'bird': 2,
 'cat': 3,
 'deer': 4,
 'dog': 5,
 'frog': 6,
 'horse': 7,
 'ship': 8,
 'truck': 9}

In [ ]:
from transformers import ViTForImageClassification

model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k',
                                                  num_labels = 10,
                                                  id2label = id2label,
                                                  label2id = label2id)

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.config

ViTConfig {
  "architectures": [
    "ViTModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "dtype": "float32",
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "airplane",
    "1": "automobile",
    "2": "bird",
    "3": "cat",
    "4": "deer",
    "5": "dog",
    "6": "frog",
    "7": "horse",
    "8": "ship",
    "9": "truck"
  },
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "airplane": 0,
    "automobile": 1,
    "bird": 2,
    "cat": 3,
    "deer": 4,
    "dog": 5,
    "frog": 6,
    "horse": 7,
    "ship": 8,
    "truck": 9
  },
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": true,
  "transformers_version": "4.56.2"
}

# Trainer 실행을 위해 필요한 아규먼트 설정

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir = "test-cifar-10", # 모델 예측과 체크포인트가 저장되는 폴더명
    save_strategy = "epoch", # epoch마다 모델 학습 전략
    eval_strategy = "epoch", # evaluation 시, epoch마다 모델 학습 전략
    learning_rate = 2e-5, # 0.00002
    per_device_train_batch_size = 16, # CPU/GPU 당 mini-batch 사이즈
    per_device_eval_batch_size = 16, # evaluation 시, CPU/GPU 당 mini-batch 사이즈
    num_train_epochs = 10, # 총 학습 에포크
    weight_decay = 0.01, # optimizer에 들어갈 weight decay
    load_best_model_at_end = True, # 학습 종료시 자동으로 베스트 모델을 로드한다.
    metric_for_best_model = "accuracy", # 베스트 모델 측정을 위한 메트릭(정확도)
    logging_dir = "logs", # Tensorboard를 위한 logs를 저장할 폴더명
    report_to = "none", # wandb 자동 로깅 비활성화
    remove_unused_columns = False, # 자동으로 모델에서 쓰지않는 컬럼 삭제 여부
    optim = "adamw_torch", # pytorch에서 제공하는 AdamW optimizer 사용
    lr_scheduler_type = "constant", # learning rate scheduler, 고정시 학습률 고정(default: linear)
    save_total_limit = 10 # 저장할 checkpoints 최대 갯수(저장용량 초과 에러 방지)
)

# Metric 설정

In [ ]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
  # 예측(검증)값과 실제값을 가져온다.
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  # 예측값 중 가장 높은 값의 인덱스 번호를 저장
  return metric.compute(predictions=predictions, references=labels)

# Trainer 정의 및 실행

In [ ]:
import torch

trainer = Trainer(
    model = model,
    args = args,
    train_dataset = train_ds,
    eval_dataset = val_ds,
    data_collator = collate_fn,
    compute_metrics = compute_metrics,
    tokenizer = feature_extractor
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.660980,0.940000
2,0.958200,0.289969,0.966000
3,0.958200,0.203597,0.956000
4,0.152000,0.170226,0.964000


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.660980,0.940000
2,0.958200,0.289969,0.966000
3,0.958200,0.203597,0.956000
4,0.152000,0.170226,0.964000
5,0.152000,0.167841,0.960000
6,0.051400,0.168710,0.962000
7,0.051400,0.156244,0.966000
8,0.029400,0.162407,0.962000
9,0.016000,0.164899,0.964000
10,0.016000,0.166887,0.962000


TrainOutput(global_step=2820, training_loss=0.21528698053765805, metrics={'train_runtime': 1734.2926, 'train_samples_per_second': 25.947, 'train_steps_per_second': 1.626, 'total_flos': 3.48738956568576e+18, 'train_loss': 0.21528698053765805, 'epoch': 10.0})

# Evaluation

In [ ]:
outputs = trainer.predict(test_ds)

In [ ]:
print(outputs.metrics)

{'test_loss': 0.28015777468681335, 'test_accuracy': 0.971, 'test_runtime': 26.4441, 'test_samples_per_second': 75.631, 'test_steps_per_second': 4.727}
